In [1]:
# This is for Puffer
# import os
# os.environ['https_proxy'] = 'http://192.41.170.23:3128'
# os.environ['http_proxy'] = 'http://192.41.170.23:3128'

In [2]:
import pandas as pd
import torch
from transformers import DPRContextEncoder, DPRContextEncoderTokenizer
import time

context_encoder = DPRContextEncoder.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")
context_tokenizer = DPRContextEncoderTokenizer.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
context_encoder.to(device)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizer'.


DPRContextEncoder(
  (ctx_encoder): DPREncoder(
    (bert_model): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bia

In [4]:
import pandas as pd
#checking is reviewId present in both csv files
df_review = pd.read_csv('train-targets.csv')
df_targets = pd.read_csv('train-input-filtered-review-30.csv')

target_review_ids = df_targets['ReviewID'].tolist()


filtered_df_review = df_review[df_review['ReviewID'].isin(target_review_ids)]

filtered_df_review.to_csv('filtered_train-targets.csv', index=False)


In [5]:
import pandas as pd

#checking out number of background and abstract, We have selected 11142 total of reviewID, 
df_original = pd.read_csv('/home/ayushk/work/Thesis/latest_script/train-input-filtered-review-30.csv')
df_filtered = pd.read_csv('/home/ayushk/work/Thesis/latest_script/filtered_train-targets.csv')

# Count unique ReviewIDs before and after filtering
unique_review_ids_original = df_original['ReviewID'].nunique()
unique_review_ids_filtered = df_filtered['ReviewID'].nunique()

print(f"Unique ReviewIDs in original data: {unique_review_ids_original}")
print(f"Unique ReviewIDs after filtering: {unique_review_ids_filtered}")


Unique ReviewIDs in original data: 11142
Unique ReviewIDs after filtering: 11142


In [1]:
import os
# os.environ['https_proxy'] = 'http://192.41.170.23:3128'
# os.environ['http_proxy'] = 'http://192.41.170.23:3128'

# Taking out Canidate document per ReviewID based on Given Background, for these we have used Dense Passage Retreival model. #we have selected top=3 document
import pandas as pd
import torch
from transformers import DPRContextEncoder, DPRContextEncoderTokenizer
import time

context_encoder = DPRContextEncoder.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")
context_tokenizer = DPRContextEncoderTokenizer.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
context_encoder.to(device)

sample_train_inputs = pd.read_csv('/home/ayushk/work/Thesis/latest_script/train-input-filtered-review-30.csv')
train_target_df = pd.read_csv('/home/ayushk/work/Thesis/latest_script/cleaned_train-targets.csv')

timings = {}

desired_output = []
batch_size = 5
for review_id in sample_train_inputs['ReviewID'].unique():
    start_time = time.time()
    review_inputs = sample_train_inputs[sample_train_inputs['ReviewID'] == review_id]
    review_abstracts = review_inputs['Abstract'].tolist()
    #print(f"Retrieval scores generated for Review ID {review_id} and {len(review_abstracts)} document")
    retrieval_scores_list = []
    batch_abstracts = [] 
    for abstract in review_abstracts:
        batch_abstracts.append(abstract)
        if len(batch_abstracts) == batch_size: 
            abstract_tokens = context_tokenizer(batch_abstracts, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
            multi_document_embeddings = context_encoder(**abstract_tokens).pooler_output
            
            background_data = train_target_df[train_target_df['ReviewID'] == review_id]['Background'].tolist()
            background_tokens = context_tokenizer(background_data, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
            background_embeddings = context_encoder(**background_tokens).pooler_output

            retrieval_scores = torch.mm(multi_document_embeddings, background_embeddings.T)
            retrieval_scores_list.append(retrieval_scores)
            
            batch_abstracts = [] 

    if batch_abstracts:
        abstract_tokens = context_tokenizer(batch_abstracts, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
        multi_document_embeddings = context_encoder(**abstract_tokens).pooler_output
        retrieval_scores = torch.mm(multi_document_embeddings, background_embeddings.T)
        retrieval_scores_list.append(retrieval_scores)
    
    retrieval_scores = torch.cat(retrieval_scores_list)
    end_time = time.time()
    elapsed_time = end_time - start_time
    timings[review_id] = elapsed_time
    print(f"Retrieval scores generated for Review ID {review_id} in {elapsed_time:.2f} seconds  of {len(review_abstracts)} document")

    k = min(3, retrieval_scores.shape[0])
    top_k_indices = torch.argsort(retrieval_scores.view(-1), descending=True)[:k]
    top_k_candidates = []

    for idx in top_k_indices:
        idx = idx.item() 
        top_k_candidate_pmids = sample_train_inputs[sample_train_inputs['ReviewID'] == review_id].iloc[idx]['PMID']
        top_k_candidate_document = sample_train_inputs[sample_train_inputs['ReviewID'] == review_id].iloc[idx]['Abstract']
        top_k_candidate_background = train_target_df[train_target_df['ReviewID'] == review_id]['Background'].values[0]

        top_k_candidates.append({
            "Document": top_k_candidate_document,
            "Background": top_k_candidate_background,
            "PMID": top_k_candidate_pmids
        })

    pmids = [doc['PMID'] for doc in top_k_candidates]
    background = top_k_candidates[0]['Background'] 
    abstracts = [doc['Document'] for doc in top_k_candidates]

    desired_output.append({
        'ReviewID': review_id,
        'PMID': pmids,
        'Background': background,
        'Candidate document': abstracts
    })

desired_output_df = pd.DataFrame(desired_output)
desired_output_df.to_csv("/home/ayushk/work/Thesis/latest_script/desired_output_final.csv", index=False)

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizer'.


Retrieval scores generated for Review ID 30760312 in 0.42 seconds  of 15 document
Retrieval scores generated for Review ID 19588356 in 0.07 seconds  of 5 document
Retrieval scores generated for Review ID 23893797 in 0.10 seconds  of 10 document
Retrieval scores generated for Review ID 27167891 in 0.10 seconds  of 10 document
Retrieval scores generated for Review ID 28753768 in 0.05 seconds  of 5 document
Retrieval scores generated for Review ID 24417824 in 0.15 seconds  of 18 document
Retrieval scores generated for Review ID 28416626 in 0.05 seconds  of 5 document
Retrieval scores generated for Review ID 26949760 in 0.24 seconds  of 26 document
Retrieval scores generated for Review ID 25299481 in 0.15 seconds  of 15 document
Retrieval scores generated for Review ID 25506813 in 0.25 seconds  of 28 document
Retrieval scores generated for Review ID 25947646 in 0.06 seconds  of 8 document
Retrieval scores generated for Review ID 24895945 in 0.34 seconds  of 29 document
Retrieval scores gen

In [3]:
#Removing  duplicate backgroun from train-targets ReviewID
import pandas as pd
df_targets = pd.read_csv('/home/ayushk/work/Thesis/latest_script/filtered_train-targets.csv')

df_targets_cleaned = df_targets.drop_duplicates(subset=['ReviewID'], keep='first')

df_targets_cleaned.to_csv('cleaned_train-targets.csv', index=False)


In [4]:
#EDA
#checking total number of ReviewID, how many multi-document do we have per each ReviewID?
import pandas as pd
df = pd.read_csv(r"/home/st122802/work/Thesis/Untitled Folder/train-input.csv")
review_id_counts = df['ReviewID'].value_counts()

review_ids_greater_than_15 = review_id_counts[review_id_counts > 15]

print("ReviewIDs with count greater than 15:")
print(review_ids_greater_than_15)
total_review_ids = len(review_ids_greater_than_15)
print(f"Total number of ReviewIDs with count greater than 15: {total_review_ids}")


ReviewIDs with count greater than 15:
ReviewID
25412402    401
23126607    398
26250104    390
28402085    389
32428300    356
           ... 
30561133     16
28450123     16
26923462     16
28669754     16
26896865     16
Name: count, Length: 7387, dtype: int64
Total number of ReviewIDs with count greater than 15: 7387
